In [1]:
!pip install -r requirements.txt

  Using cached pluggy-1.5.0-py3-none-any.whl.metadata (4.8 kB)
Using cached pluggy-1.5.0-py3-none-any.whl (20 kB)
  Attempting uninstall: pluggy
    Found existing installation: pluggy 1.3.0
    Not uninstalling pluggy at /shared-libs/python3.11/py-core/lib/python3.11/site-packages, outside environment /root/venv
    Can't uninstall 'pluggy'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [26]:
import duckdb
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats
import statsmodels.api as sm

In [2]:
conn = duckdb.connect()

In [3]:
conn.execute("CREATE VIEW ratings AS SELECT * FROM read_parquet('/work/Dataset/BeerAdvocate/Processed/ratings.pq') WHERE date is NOT NULL;")

In [4]:
ratings_py = conn.execute("SELECT date, rating, style, abv, beer_id, brewery_name FROM ratings")

In [5]:
info = conn.execute("SELECT * FROM ratings limit 10").df()
info['date'] = pd.to_datetime(info['date'])
print(info.info())
display(info)
del info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       10 non-null     object        
 1   rating        10 non-null     float64       
 2   review        10 non-null     bool          
 3   abv           10 non-null     float64       
 4   brewery_name  10 non-null     object        
 5   user_name     10 non-null     object        
 6   beer_id       10 non-null     int64         
 7   appearance    5 non-null      float64       
 8   palate        5 non-null      float64       
 9   text          5 non-null      object        
 10  aroma         5 non-null      float64       
 11  overall       5 non-null      float64       
 12  taste         5 non-null      float64       
 13  style         10 non-null     object        
 14  beer_name     10 non-null     object        
 15  brewery_id    10 non-null     int64        

,user_id,rating,review,abv,brewery_name,user_name,beer_id,appearance,palate,text,aroma,overall,taste,style,beer_name,brewery_id,date
0,nmann08.184925,2.88,True,4.5,Societe des Brasseries du Gabon (SOBRAGA),nmann08,142544,3.25,3.25,"From a bottle, pours a piss yellow color with ...",2.75,3.0,2.75,Euro Pale Lager,Régab,37262,2015-08-20 12:00:00
1,stjamesgate.163714,3.67,True,4.5,Strangford Lough Brewing Company Ltd,StJamesGate,19590,3.00,3.50,Pours pale copper with a thin head that quickl...,3.50,3.5,4.00,English Pale Ale,Barelegs Brew,10093,2009-02-20 12:00:00
2,mdagnew.19527,3.73,True,4.5,Strangford Lough Brewing Company Ltd,mdagnew,19590,4.00,3.50,"500ml Bottle bought from The Vintage, Antrim.....",3.50,3.5,4.00,English Pale Ale,Barelegs Brew,10093,2006-03-13 12:00:00
3,helloloser12345.10867,3.98,True,4.5,Strangford Lough Brewing Company Ltd,helloloser12345,19590,4.00,4.00,Serving: 500ml brown bottlePour: Good head wit...,3.50,4.5,4.00,English Pale Ale,Barelegs Brew,10093,2004-12-01 12:00:00
4,cypressbob.3708,4.00,True,4.5,Strangford Lough Brewing Company Ltd,cypressbob,19590,4.00,4.00,"500ml bottlePours with a light, slightly hazy ...",4.00,4.0,4.00,English Pale Ale,Barelegs Brew,10093,2004-08-30 12:00:00
5,hellpop65.48993,3.25,False,4.8,Strangford Lough Brewing Company Ltd,Hellpop65,19827,NaN,NaN,None,NaN,NaN,NaN,English Pale Ale,Legbiter,10093,2014-12-01 12:00:00
6,latarnik.52897,3.50,False,4.8,Strangford Lough Brewing Company Ltd,Latarnik,19827,NaN,NaN,None,NaN,NaN,NaN,English Pale Ale,Legbiter,10093,2014-05-29 12:00:00
7,rochefortchris.697017,3.50,False,4.8,Strangford Lough Brewing Company Ltd,RochefortChris,19827,NaN,NaN,None,NaN,NaN,NaN,English Pale Ale,Legbiter,10093,2014-02-26 12:00:00
8,okcnittany.144868,3.75,False,4.8,Strangford Lough Brewing Company Ltd,OKCNittany,19827,NaN,NaN,None,NaN,NaN,NaN,English Pale Ale,Legbiter,10093,2014-02-10 12:00:00
9,jaydoc.265507,3.25,False,4.8,Strangford Lough Brewing Company Ltd,jaydoc,19827,NaN,NaN,None,NaN,NaN,NaN,English Pale Ale,Legbiter,10093,2014-01-25 12:00:00


In [6]:
conn.execute("""
CREATE OR REPLACE VIEW monthly_ratings AS (
    WITH monthly_ratings2 AS (
    SELECT 
        DATE_TRUNC('month', date) AS date,
        AVG(rating) as avg_rating,
        COUNT(rating) as rating_count,
        STDDEV(rating) as rating_std,
        AVG(abv) as avg_abv,
        STDDEV(abv) as abv_std,
        COUNT(DISTINCT beer_id) as unique_beers
    FROM ratings
    GROUP BY DATE_TRUNC('month', date)
    )
    SELECT 
        date,
        avg_rating,
        rating_count,
        rating_std,
        avg_abv,
        abv_std,
        unique_beers,
    EXTRACT(year FROM date) as year,
    EXTRACT(month FROM date) as month
    FROM monthly_ratings2
    ORDER BY date
);
""")

In [7]:
monthly_info = conn.execute("SELECT * FROM monthly_ratings LIMIT 10").df()
print(monthly_info.info())
display(monthly_info)
del monthly_info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          10 non-null     datetime64[us]
 1   avg_rating    10 non-null     float64       
 2   rating_count  10 non-null     int64         
 3   rating_std    9 non-null      float64       
 4   avg_abv       10 non-null     float64       
 5   abv_std       9 non-null      float64       
 6   unique_beers  10 non-null     int64         
 7   year          10 non-null     int64         
 8   month         10 non-null     int64         
dtypes: datetime64[us](1), float64(4), int64(4)
memory usage: 852.0 bytes
None


,date,avg_rating,rating_count,rating_std,avg_abv,abv_std,unique_beers,year,month
0,1996-08-01,3.850000,1,NaN,5.300000,NaN,1,1996,8
1,1998-01-01,3.598000,5,1.135768,9.100000,7.275301,5,1998,1
2,1998-02-01,3.626667,18,0.435498,6.007500,1.504896,18,1998,2
3,1998-03-01,3.792308,13,0.443455,5.287500,0.977515,13,1998,3
4,1998-04-01,3.864286,7,0.512961,5.772000,1.713161,7,1998,4
5,1998-05-01,3.750000,18,0.447608,6.010000,1.829795,18,1998,5
6,1998-06-01,3.931429,14,0.620395,6.540000,1.690562,14,1998,6
7,1998-07-01,3.795556,9,0.510444,5.900000,0.738241,9,1998,7
8,1998-08-01,3.894706,17,0.518569,6.353846,1.850052,17,1998,8
9,1998-09-01,3.900714,14,0.562815,5.037500,1.262580,14,1998,9


In [8]:
monthly_df = conn.execute("SELECT date, avg_rating, rating_count FROM monthly_ratings ORDER BY date").df()
monthly_df['date'] = pd.to_datetime(monthly_df['date'])

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
   go.Scatter(x=monthly_df['date'],
              y=monthly_df['avg_rating'],
              name="Average Rating",
              line=dict(color='blue')),
   secondary_y=False
)

fig.add_trace(
   go.Scatter(x=monthly_df['date'],
              y=monthly_df['rating_count'], 
              name="Number of Reviews",
              line=dict(color='red')),
   secondary_y=True
)

fig.update_layout(
   title="Beer Rating Trends Over Time",
   xaxis_title="Date",
   yaxis_title="Average Rating", 
   yaxis2_title="Number of Reviews"
)

fig.show()
del monthly_df

In [12]:
monthly_df = conn.execute("SELECT * FROM monthly_ratings ORDER BY date").df()
monthly_df['date'] = pd.to_datetime(monthly_df['date'])

ts_data = monthly_df.set_index('date')['avg_rating']
ts_data_clean = ts_data.ffill().bfill()

decomposition = seasonal_decompose(ts_data_clean, period=12)

fig = make_subplots(rows=4, cols=1, 
                   subplot_titles=("Original", "Trend", "Seasonal", "Residual"))

fig.add_trace(go.Scatter(x=ts_data_clean.index, y=ts_data_clean.values,
                        name="Original"), row=1, col=1)
fig.add_trace(go.Scatter(x=ts_data_clean.index, y=decomposition.trend,
                        name="Trend"), row=2, col=1)
fig.add_trace(go.Scatter(x=ts_data_clean.index, y=decomposition.seasonal,
                        name="Seasonal"), row=3, col=1)
fig.add_trace(go.Scatter(x=ts_data_clean.index, y=decomposition.resid,
                        name="Residual"), row=4, col=1)

fig.update_layout(height=900,
                 title_text="Seasonal Decomposition of Beer Ratings",
                 showlegend=True)

fig.update_yaxes(title_text="Rating", row=1, col=1)
fig.update_yaxes(title_text="Trend", row=2, col=1)
fig.update_yaxes(title_text="Seasonal", row=3, col=1)
fig.update_yaxes(title_text="Residual", row=4, col=1)

fig.show()

In [14]:
seasonal_patterns = pd.DataFrame({
    'month': range(1, 13),
    'seasonal_effect': decomposition.seasonal[:12]
})

seasonal_patterns['month_name'] = pd.date_range(start='2024-01-01', periods=12, freq='M').strftime('%B')

fig = go.Figure()
fig.add_trace(go.Bar(
    x=seasonal_patterns['month_name'],
    y=seasonal_patterns['seasonal_effect'],
    name='Monthly Effect'
))

fig.update_layout(
    title="Monthly Effect on Beer Ratings by Month",
    xaxis_title="Month",
    yaxis_title="Rating Effect",
    showlegend=True
)

fig.add_hline(y=0, line_dash="dash", line_color="red")

fig.show()

In [15]:
top_styles_py = conn.execute("""
   SELECT style, COUNT(*) as count 
   FROM ratings 
   GROUP BY style 
   ORDER BY count DESC 
   LIMIT 10
""").df()

print("Top 10 Beer Styles:")
top_styles_py

Top 10 Beer Styles:


,style,count
0,American IPA,972958
1,American Double / Imperial IPA,721616
2,American Double / Imperial Stout,496850
3,American Pale Ale (APA),372509
4,Saison / Farmhouse Ale,296493
5,American Wild Ale,266715
6,American Porter,252174
7,Russian Imperial Stout,250008
8,American Amber / Red Ale,197827
9,Fruit / Vegetable Beer,162270


In [17]:
style_trends = conn.execute("""
   WITH top_styles AS (
       SELECT style, COUNT(*) as cnt
       FROM ratings
       GROUP BY style
       ORDER BY cnt DESC
       LIMIT 10
   )
   SELECT 
       r.style,
       date_trunc('year', r.date) as date,
       AVG(r.rating) as avg_rating,
       COUNT(*) as rating_count
   FROM ratings r
   INNER JOIN top_styles t ON t.style = r.style
   GROUP BY r.style, date_trunc('year', r.date)
   ORDER BY date, r.style
""").df()

style_trends['date'] = pd.to_datetime(style_trends['date'])

fig = px.line(style_trends, x='date', y='avg_rating', color='style',
             title='Rating Trends by Beer Style')
fig.show()

In [23]:
def perform_mann_kendall(data):
    """
    Perform Mann-Kendall trend test with proper error handling
    """
    try:
        time_idx = range(len(data))

        valid_pairs = [(t, x) for t, x in zip(time_idx, data) if not np.isnan(x)]
        if not valid_pairs:
            return None, None

        times, values = zip(*valid_pairs)

        mk_result = stats.kendalltau(times, values)

        return mk_result.correlation, mk_result.pvalue

    except Exception as e:
        print(f"Error in Mann-Kendall test: {str(e)}")
        return None, None

monthly_df = conn.execute("""
    WITH filled AS (
        SELECT *,
            avg_rating as cleaned_avg_rating,
            ROW_NUMBER() OVER (ORDER BY date) as rn
        FROM monthly_ratings
        WHERE avg_rating IS NOT NULL
    )
    SELECT 
        m.*,
        COALESCE(m.avg_rating, 
            (SELECT cleaned_avg_rating 
             FROM filled f 
             WHERE f.rn = (
                 SELECT MAX(rn) 
                 FROM filled 
                 WHERE filled.date < m.date
             )),
            (SELECT cleaned_avg_rating 
             FROM filled f 
             WHERE f.rn = (
                 SELECT MIN(rn) 
                 FROM filled 
                 WHERE filled.date > m.date
             ))
        ) as cleaned_avg_rating
    FROM monthly_ratings m
    ORDER BY date
""").df()

tau, p_value = perform_mann_kendall(monthly_df['cleaned_avg_rating'])

print("\nMann-Kendall Trend Test Results:")
if tau is not None and p_value is not None:
    print(f"Correlation (tau): {tau:.4f}")
    print(f"P-value: {p_value:.4f}")
    print(f"Trend interpretation: {'Significant' if p_value < 0.05 else 'Not significant'} "
          f"{'positive' if tau > 0 else 'negative'} trend")
else:
    print("Unable to perform Mann-Kendall test due to data issues")


Mann-Kendall Trend Test Results:
Correlation (tau): 0.4005
P-value: 0.0000
Trend interpretation: Significant positive trend


In [24]:
def perform_linear_trend_analysis(df):
    time_idx = np.arange(len(df))

    slope, intercept, r_value, p_value, std_err = stats.linregress(time_idx,
                                                                   df['avg_rating'])
    return {
        'slope': slope,
        'intercept': intercept,
        'r_squared': r_value**2,
        'p_value': p_value,
        'std_err': std_err
    }

monthly_df = conn.execute("""
   WITH filled AS (
       SELECT *,
           avg_rating as cleaned_avg_rating,
           ROW_NUMBER() OVER (ORDER BY date) as rn
       FROM monthly_ratings
       WHERE avg_rating IS NOT NULL
   )
   SELECT 
       m.*,
       COALESCE(m.avg_rating, 
           (SELECT cleaned_avg_rating 
            FROM filled f 
            WHERE f.rn = (
                SELECT MAX(rn) 
                FROM filled 
                WHERE filled.date < m.date
            )),
           (SELECT cleaned_avg_rating 
            FROM filled f 
            WHERE f.rn = (
                SELECT MIN(rn) 
                FROM filled 
                WHERE filled.date > m.date
            ))
       ) as cleaned_avg_rating
   FROM monthly_ratings m
   ORDER BY date
""").df()

trend_results = perform_linear_trend_analysis(monthly_df)

print("\nLinear Trend Analysis Results:")
print(f"Slope: {trend_results['slope']:.6f} (change in rating per month)")
print(f"R-squared: {trend_results['r_squared']:.4f}")
print(f"P-value: {trend_results['p_value']:.4f}")
print(f"Standard Error: {trend_results['std_err']:.6f}")


Linear Trend Analysis Results:
Slope: 0.000507 (change in rating per month)
R-squared: 0.0717
P-value: 0.0000
Standard Error: 0.000120


In [28]:
data = conn.execute("""
   WITH filled AS (
       SELECT 
           date,
           COALESCE(avg_rating, 
               LAG(avg_rating) OVER (ORDER BY date),
               LEAD(avg_rating) OVER (ORDER BY date)
           ) as cleaned_avg_rating
       FROM monthly_ratings
   )
   SELECT cleaned_avg_rating
   FROM filled
   ORDER BY date
""").df()

def plot_autocorrelation(data, lags=24):
   acf = sm.tsa.stattools.acf(data['cleaned_avg_rating'], nlags=lags, fft=False)
   conf_int = 1.96/np.sqrt(len(data))
   
   fig = go.Figure()
   fig.add_trace(go.Bar(
       x=list(range(len(acf))),
       y=acf,
       name='Autocorrelation'
   ))
   
   fig.add_hline(y=conf_int, line_dash="dash", line_color="red",
                 annotation_text="95% Confidence Interval")
   fig.add_hline(y=-conf_int, line_dash="dash", line_color="red")
   
   fig.update_layout(
       title="Autocorrelation Plot with Confidence Intervals",
       xaxis_title="Lag (months)",
       yaxis_title="Correlation",
       showlegend=True
   )
   
   return fig

acf_fig = plot_autocorrelation(data)
acf_fig.show()

In [29]:
def test_seasonality(data, period=12):
    """
    Test for the presence of seasonality using various methods
    """
    # Prepare seasonal means
    seasonal_means = []
    for i in range(period):
        seasonal_means.append(data[i::period].mean())

    # Perform Kruskal-Wallis H-test
    seasons = []
    for i in range(period):
        seasons.append(data[i::period])

    h_stat, p_value = stats.kruskal(*seasons)

    return {
        'seasonal_means': seasonal_means,
        'h_statistic': h_stat,
        'p_value': p_value
    }

# Test for seasonality
season_test = test_seasonality(data['cleaned_avg_rating'].values)

print("\nSeasonality Test Results (Kruskal-Wallis H-test):")
print(f"H-statistic: {season_test['h_statistic']:.4f}")
print(f"P-value: {season_test['p_value']:.4f}")
print("Interpretation: Seasonal differences are",
      "significant" if season_test['p_value'] < 0.05 else "not significant")


Seasonality Test Results (Kruskal-Wallis H-test):
H-statistic: 11.5789
P-value: 0.3961
Interpretation: Seasonal differences are not significant


In [30]:
print("Statistical Analysis Summary:")
print("-" * 50)
print("\n1. Trend Analysis:")
if tau is not None:
    print(f"- Mann-Kendall tau: {tau:.4f} ({'significant' if p_value < 0.05 else 'not significant'})")
print(f"- Linear trend: {trend_results['slope']:.6f} per month (R² = {trend_results['r_squared']:.4f})")

print("\n2. Seasonality:")
print(f"- Seasonal effect: {'present' if season_test['p_value'] < 0.05 else 'not detected'}")
print(f"- Seasonality p-value: {season_test['p_value']:.4f}")

print("\n3. Additional Insights:")
if trend_results['p_value'] < 0.05:
    print(f"- Rating change per year: {trend_results['slope']*12:.4f}")
    print(f"- Projected change over 5 years: {trend_results['slope']*12*5:.4f}")

Statistical Analysis Summary:
--------------------------------------------------

1. Trend Analysis:
- Mann-Kendall tau: 0.4005 (significant)
- Linear trend: 0.000507 per month (R² = 0.0717)

2. Seasonality:
- Seasonal effect: not detected
- Seasonality p-value: 0.3961

3. Additional Insights:
- Rating change per year: 0.0061
- Projected change over 5 years: 0.0304


In [37]:
monthly_df = conn.execute("SELECT * FROM monthly_ratings ORDER BY date").df()
monthly_df['date'] = pd.to_datetime(monthly_df['date'])

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=monthly_df['date'],
               y=monthly_df['avg_abv'],
               name="Average ABV",
               line=dict(color='green')),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=monthly_df['date'],
               y=monthly_df['rating_count'],
               name="Number of Reviews",
               line=dict(color='gray', dash='dot')),
    secondary_y=True
)

fig.add_trace(
    go.Scatter(x=monthly_df['date'],
               y=monthly_df['unique_beers'],
               name="Number of Unique Beers",
               line=dict(color='blue', dash='dot')),
    secondary_y=True
)

fig.update_layout(
    title="Beer ABV and Volume Trends Over Time", 
    xaxis_title="Date",
    yaxis_title="Average ABV",
    yaxis2_title="Count (Reviews & Beers)"
)

fig.show()

In [48]:
monthly_df_cum = conn.execute("""
   WITH monthly_stats AS (
       SELECT 
           date_trunc('month', date) as date,
           AVG(abv) as avg_abv,
           COUNT(*) as rating_count,
           COUNT(DISTINCT beer_id) as unique_beers_that_month
       FROM ratings 
       GROUP BY date_trunc('month', date)
   ),
   cumulative AS (
       SELECT 
           date,
           avg_abv,
           rating_count,
           SUM(unique_beers_that_month) OVER (ORDER BY date) as cumulative_unique_beers,
           SUM(rating_count) OVER (ORDER BY date) as cumulative_ratings
       FROM monthly_stats
   )
   SELECT * FROM cumulative
   ORDER BY date
""").df()

monthly_df_cum['date'] = pd.to_datetime(monthly_df_cum['date'])


In [49]:
window_size = 12 
monthly_df_cum['abv_trend'] = monthly_df_cum['avg_abv'].rolling(window=window_size, center=True).mean()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
   go.Scatter(x=monthly_df_cum['date'],
              y=monthly_df_cum['avg_abv'],
              name="Average ABV",
              line=dict(color='green')),
   secondary_y=False
)

fig.add_trace(
   go.Scatter(x=monthly_df_cum['date'],
              y=monthly_df_cum['abv_trend'],
              name="ABV 12-month Moving Average",
              line=dict(color='red')),
   secondary_y=False
)

fig.add_trace(
   go.Scatter(x=monthly_df_cum['date'],
              y=monthly_df_cum['cumulative_ratings'],
              name="Cumulative Reviews",
              line=dict(color='gray', dash='dot')),
   secondary_y=True
)

fig.add_trace(
   go.Scatter(x=monthly_df_cum['date'],
              y=monthly_df_cum['cumulative_unique_beers'],
              name="Cumulative Unique Beers",
              line=dict(color='blue', dash='dot')),
   secondary_y=True
)

fig.update_layout(
   title="Beer ABV and Cumulative Volume Trends Over Time", 
   xaxis_title="Date",
   yaxis_title="Average ABV",
   yaxis2_title="Cumulative Count (Reviews & Beers)"
)

fig.show()

In [36]:
# Calculate summary statistics
summary_stats = {
    'Total Time Period': f"{monthly_df['date'].min().year} to {monthly_df['date'].max().year}",
    'Total Reviews': monthly_df['rating_count'].sum(),
    'Average Rating': monthly_df['avg_rating'].mean(),
    'Rating Trend (tau)': tau,
    'Trend Significance': 'Significant' if p_value < 0.05 else 'Not Significant',
    'Average ABV': monthly_df['avg_abv'].mean(),
    'Unique Beers Reviewed': monthly_df['unique_beers'].sum()
}

# Display summary
for key, value in summary_stats.items():
    print(f"{key}: {value}")

Total Time Period: 1996 to 2017
Total Reviews: 8229373
Average Rating: 3.8263355333002913
Rating Trend (tau): 0.4004768894844279
Trend Significance: Significant
Average ABV: 6.802879015108632
Unique Beers Reviewed: 1861834


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=68ca1048-0261-4e0d-b190-606d96d6d84c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>